In [2]:
#install folium
!pip install folium==0.5.0 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 12.3 MB/s eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=1e223310c9878893c82e2210a731163ce9e38869246dbdb02db9b9b32550ec64
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [3]:
#Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
import json 
from pandas.io.json import json_normalize 
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<h1>Part 1<h1>
<p>For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.</p>

<p><b>Get the csv file and save it to the soap object</b></p>

In [4]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") # Wikipedia url page 
soup = BeautifulSoup(html_data.content,'html5lib') # Pass the url to soup object 

<p><b>It extracts the information from the table and saves it in dataframe</b></p>

In [6]:
#Extract information from the web page table.The information will be saved in a dataframe object 
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<p><b>the number of rows in the df is:</b></p>

In [7]:
df.shape[0]

103

<h1>Part 2<h1>
<p>Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.</p>

<p><b>The location data is obtained</b></p>

In [8]:
#Due to problems using geopy the csv file is imported 
df_coordinates = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
df_coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<p><b>The two dataframes are joined</b></p>

In [9]:
#Merge dataframes
df_coordinates.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_united = pd.merge(df, df_coordinates, on='PostalCode')
df_united

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h1>Part 3<h1>
<p>Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.</p>

<p><b>Print number of boroughs and neighborhoods</b></p>

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_united['Borough'].unique()),
        df_united.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


<p><b>Get location of Toronto, Canada(latitude and longitude)</b></p>

In [11]:
address = 'toronto,canada'

geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<p><b>Create toronto map and add each of the neighborhoods</b></p>

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_united['Latitude'], df_united['Longitude'], df_united['Borough'], df_united['Neighborhood']):
    label = '{}--{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<p><b>I get the neighborhoods of downtown Toronto</b></p>

In [13]:
downtown_toronto_data = df_united[df_united['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


<p><b>I get the location of downtown toronto</b></p>

In [14]:
address = 'Downtown, toronto'

geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081162653639.


<p><b>Create the map of toronto and center the location in the center of the city.</b></p>

In [19]:
# create map of Downtown using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<p><b>Create foursquare credentials.</b></p>

In [20]:
CLIENT_ID = 'OZXNGO4EF05404GACE2FC51TO4NJMM10R4BP10JUQQUVHTCZ' # your Foursquare ID
CLIENT_SECRET = 'TRH3FCC0E0DUXBDMBOWHCAIUGHRO23YKJ0MVQG5I541BYRHZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OZXNGO4EF05404GACE2FC51TO4NJMM10R4BP10JUQQUVHTCZ
CLIENT_SECRET:TRH3FCC0E0DUXBDMBOWHCAIUGHRO23YKJ0MVQG5I541BYRHZ


<p><b>I do the study of the neighborhoods: Garden District, Ryerson of downtown toronto.</b></p>

In [21]:
downtown_toronto_data.loc[1, 'Neighborhood']

'Garden District, Ryerson'

<p><b>Get the location of the neighborhoods: Garden District, Ryerson downtown Toronto.</b></p> 

In [22]:
neighborhood_latitude = downtown_toronto_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_toronto_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_toronto_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Garden District, Ryerson are 43.6571618, -79.37893709999999.


<p><b>I get locations within 200 yards of neighborhoods: Garden District, Ryerson.</b></p> 

In [23]:
radius = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=OZXNGO4EF05404GACE2FC51TO4NJMM10R4BP10JUQQUVHTCZ&client_secret=TRH3FCC0E0DUXBDMBOWHCAIUGHRO23YKJ0MVQG5I541BYRHZ&v=20180605&ll=43.6571618,-79.37893709999999&radius=200&limit=100'

<p><b>Get the json from foursquare.</b></p> 

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60ce2d8c27addc716137f886'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Toronto',
  'headerFullLocation': 'Downtown Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 27,
  'suggestedBounds': {'ne': {'lat': 43.6589618018, 'lng': -79.3764537800555},
   'sw': {'lat': 43.6553617982, 'lng': -79.38142041994448}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5615b6c4498e3c32c67ad78f',
       'name': 'Blaze Pizza',
       'location': {'address': '10 Dundas Street East, #124',
        'lat': 43.656518,
        'lng': -79.380015,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.656518,
 

<p><b>Function that extracts the category of the venue.</b></p> 

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<p><b>I convert values from the json and save them in dataframe.</b></p> 

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Blaze Pizza,Pizza Place,43.656518,-79.380015
1,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
2,Oakham Café,Café,43.658078,-79.378315
3,Burrito Boyz,Burrito Place,43.656265,-79.378343
4,Silver Snail Comics,Comic Shop,43.657031,-79.381403
5,Milo's Pita,Falafel Restaurant,43.656270,-79.380974
6,Jazz Bistro,Music Venue,43.655678,-79.379276
7,Ali Baba's - Yonge north of Dundas,Middle Eastern Restaurant,43.657867,-79.381176
8,Yonge-Dundas Square,Plaza,43.656054,-79.380495
9,Second Cup,Coffee Shop,43.656027,-79.380575


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


<p><b>Function to get places close to neighborhoods.</b></p> 

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<p><b>Get places close to neighborhoods. </b></p> 

In [29]:
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [30]:
#print first dataframe values 
downtown_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center


<p><b>Group the dataframe by neighborhood.</b></p> 

In [31]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,1,1
Central Bay Street,12,12,12,12,12,12
Christie,1,1,1,1,1,1
Church and Wellesley,22,22,22,22,22,22
"Commerce Court, Victoria Hotel",46,46,46,46,46,46
"First Canadian Place, Underground city",34,34,34,34,34,34
"Garden District, Ryerson",27,27,27,27,27,27
"Harbourfront East, Union Station, Toronto Islands",17,17,17,17,17,17
"Kensington Market, Chinatown, Grange Park",29,29,29,29,29,29


<p><b>Number of unique categories.</b></p> 

In [32]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 90 uniques categories.


<p><b>Performs one hot encoding to column "Venue Category".</b></p> 

In [33]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Wine Bar,Adult Boutique,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,...,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theme Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
downtown_toronto_onehot.shape

(281, 90)

<p><b>Groups the data frame of the one hot encoding values with the neighborhoods.</b></p> 

In [35]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped

,Neighborhood,Wine Bar,Adult Boutique,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theme Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000
2,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Church and Wellesley,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.045455
4,"Commerce Court, Victoria Hotel",0.021739,0.000000,0.021739,0.021739,0.000000,0.021739,0.000000,0.000000,0.021739,...,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000
5,"First Canadian Place, Underground city",0.029412,0.000000,0.029412,0.029412,0.000000,0.000000,0.000000,0.029412,0.029412,...,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000
8,"Kensington Market, Chinatown, Grange Park",0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.034483
9,"Regent Park, Harbourfront",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
downtown_toronto_grouped.shape

(16, 90)

<p><b>Let's print each neighborhood along with the top 3 most common venues.</b></p> 

In [37]:
num_top_venues = 3

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                   venue  freq
0  Performing Arts Venue   1.0
1           Gourmet Shop   0.0
2                   Park   0.0


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.50
1  Sandwich Place  0.17
2        Pharmacy  0.08


----Christie----
                      venue  freq
0       Japanese Restaurant   1.0
1                  Wine Bar   0.0
2  Mediterranean Restaurant   0.0


----Church and Wellesley----
                venue  freq
0  Mexican Restaurant  0.09
1      Sandwich Place  0.09
2       Burrito Place  0.09


----Commerce Court, Victoria Hotel----
            venue  freq
0     Coffee Shop  0.13
1            Café  0.09
2  Sandwich Place  0.07


----First Canadian Place, Underground city----
            venue  freq
0     Coffee Shop  0.21
1  Sandwich Place  0.12
2            Café  0.09


----Garden District, Ryerson----
         venue  freq


<p><b>First, let's write a function to sort the venues in descending order.</b></p> 

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<p><b>Now let's create the new dataframe and display the top 15 venues for each neighborhood.</b></p> 

In [39]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Deli / Bodega,Dessert Shop,Diner,Donut Shop
1,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Pharmacy,Bookstore,Italian Restaurant,Convenience Store,Cocktail Bar,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Vietnamese Restaurant,Creperie
2,Christie,Japanese Restaurant,Vietnamese Restaurant,Deli / Bodega,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Dessert Shop,Clothing Store,Diner,Donut Shop
3,Church and Wellesley,Mexican Restaurant,Burrito Place,Sandwich Place,Gym / Fitness Center,Korean Restaurant,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Breakfast Spot,Coffee Shop,Bank,Vietnamese Restaurant,Escape Room,Dessert Shop,Adult Boutique
4,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Bank,Deli / Bodega,Sandwich Place,Hotel,Salad Place,Gym,Pharmacy,Restaurant,Gastropub,Fast Food Restaurant,Clothing Store,Gluten-free Restaurant,Japanese Restaurant


<p><b>Run k-means to cluster the neighborhood into 7 clusters.</b></p> 

In [40]:
# set number of clusters
kclusters = 7

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 3, 0, 0, 0, 0, 0, 0, 5], dtype=int32)

<p><b>Let's create a new dataframe that includes the cluster as well as the top 15 venues for each neighborhood.</b></p> 

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = downtown_toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#One of the neighborhoods could not be classified therefore it is deleted from the dataframe 
downtown_toronto_merged.dropna(inplace=True)
downtown_toronto_merged.reset_index(inplace=True,drop=True)
downtown_toronto_merged['Cluster Labels']=downtown_toronto_merged['Cluster Labels'].astype('int')#Convert a int
downtown_toronto_merged['Cluster Labels']

0     5
1     0
2     6
3     2
4     3
5     0
6     0
7     0
8     0
9     0
10    0
11    1
12    4
13    0
14    0
15    0
Name: Cluster Labels, dtype: int64

<p><b>Finally, let's visualize the resulting clusters.</b></p> 

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Each cluster generated is analyzed to observe the coincidence of its Members</h2>

In [44]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Middle Eastern Restaurant,Café,Sandwich Place,Pizza Place,Burrito Place,Pub,Ethiopian Restaurant,Diner,Comic Shop,College Rec Center,Clothing Store,Music Venue,Burger Joint,Plaza
5,Downtown Toronto,0,Steakhouse,Asian Restaurant,Opera House,Sandwich Place,Coffee Shop,Japanese Restaurant,Park,Colombian Restaurant,Breakfast Spot,Concert Hall,Bar,Greek Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Food Court
6,Downtown Toronto,0,Coffee Shop,Bank,Playground,Persian Restaurant,Sandwich Place,Bubble Tea Shop,Deli / Bodega,Sports Bar,Dessert Shop,Supermarket,Tea Room,Café,New American Restaurant,Pizza Place,Cocktail Bar
7,Downtown Toronto,0,Coffee Shop,Sandwich Place,Bank,Deli / Bodega,Gym,Bakery,Restaurant,Pharmacy,Gym / Fitness Center,Hotel,Gluten-free Restaurant,Burger Joint,Café,Chinese Restaurant,Clothing Store
8,Downtown Toronto,0,Coffee Shop,Café,Bank,Deli / Bodega,Sandwich Place,Hotel,Salad Place,Gym,Pharmacy,Restaurant,Gastropub,Fast Food Restaurant,Clothing Store,Gluten-free Restaurant,Japanese Restaurant
9,Downtown Toronto,0,Sandwich Place,Coffee Shop,College Gym,Pizza Place,BBQ Joint,Convenience Store,Korean Restaurant,Creperie,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Vietnamese Restaurant,Clothing Store,Deli / Bodega
10,Downtown Toronto,0,Café,Caribbean Restaurant,Vietnamese Restaurant,Dessert Shop,Mexican Restaurant,Martial Arts School,Liquor Store,Gourmet Shop,Vegetarian / Vegan Restaurant,Fish & Chips Shop,Farmers Market,Donut Shop,Coffee Shop,Organic Grocery,Cocktail Bar
13,Downtown Toronto,0,Café,Bakery,Italian Restaurant,Sri Lankan Restaurant,Pizza Place,Beer Store,Restaurant,Indian Restaurant,Coffee Shop,Diner,Dessert Shop,Concert Hall,College Gym,College Rec Center,Farmers Market
14,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Bank,Deli / Bodega,Restaurant,Gym,Gluten-free Restaurant,Gym / Fitness Center,American Restaurant,Art Gallery,Bagel Shop,Bakery,Bar,Burger Joint
15,Downtown Toronto,0,Mexican Restaurant,Burrito Place,Sandwich Place,Gym / Fitness Center,Korean Restaurant,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Breakfast Spot,Coffee Shop,Bank,Vietnamese Restaurant,Escape Room,Dessert Shop,Adult Boutique


In [45]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
11,Downtown Toronto,1,Performing Arts Venue,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Deli / Bodega,Dessert Shop,Diner,Donut Shop


In [46]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
3,Downtown Toronto,2,Coffee Shop,Sandwich Place,Sushi Restaurant,Pharmacy,Bookstore,Italian Restaurant,Convenience Store,Cocktail Bar,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Vietnamese Restaurant,Creperie


In [47]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
4,Downtown Toronto,3,Japanese Restaurant,Vietnamese Restaurant,Deli / Bodega,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Dessert Shop,Clothing Store,Diner,Donut Shop


In [48]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
12,Downtown Toronto,4,Park,Vietnamese Restaurant,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Deli / Bodega,Dessert Shop,Diner,Donut Shop


In [49]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 5, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Downtown Toronto,5,Coffee Shop,Gym / Fitness Center,Mediterranean Restaurant,Bakery,Spa,Breakfast Spot,Creperie,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Vietnamese Restaurant,Deli / Bodega


In [50]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 6, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,Downtown Toronto,6,Gastropub,Italian Restaurant,Coffee Shop,Creperie,Japanese Restaurant,Falafel Restaurant,Ethiopian Restaurant,Fish & Chips Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Fast Food Restaurant
